In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from pint import UnitRegistry

from analysis.util import dedup_trace_legends

In [2]:
units = UnitRegistry()

# noinspection PyTypeChecker
df_od = (
    pd.read_excel(
        '/Users/pasha/onedrive/nu/Katsev Pavel/04 Raw Data/EXP_0011_PK_20241029_ArgConcentrationResponse.xlsx',
        sheet_name='Day 3 - 5 OD tracking',
        usecols='A:J',
        skiprows=3,
    )
    .assign(arg_conc=lambda df: df.condition.str.split('•').str.get(0).apply(units.parse_expression))
    .assign(arg_conc_uM=lambda df: df.arg_conc.apply(lambda qty: int(qty.to('micromolar').magnitude)))
    # .assign(lim=lambda df: np.abs((df.od_sample - df.od_sample.mean()) / df.od_sample.std(ddof=0)) < 3)
)
# df_od

In [3]:
df_od_mean_sem = (
    df_od
    # remove bad inoculate 6.1
    .loc[lambda df: df.sample_id != 6.1]
    .groupby(['elapsed_time_hr', 'strain_id', 'arg_conc_uM'], sort=False)
    .agg(
        od_sample_mean=pd.NamedAgg('od_sample', 'mean'),
        od_sample_sem=pd.NamedAgg('od_sample', 'sem'),
    )
    .round(3)
    .reset_index()
)
df_od_mean_sem

,elapsed_time_hr,strain_id,arg_conc_uM,od_sample_mean,od_sample_sem
0,0.0,KSF111 + ∆argB,0,0.109,0.001
1,0.0,KSF111 + ∆argB,50,0.111,0.002
2,0.0,KSF111 + ∆argB,100,0.113,0.004
3,0.0,KSF111 + ∆argB,150,0.107,0.001
4,0.0,KSF111 + ∆argB,200,0.109,0.002
5,0.0,KSF111 + ∆argB,300,0.106,0.002
6,0.0,KSF001 wt,0,0.120,NaN
7,0.0,KSF001 wt,300,0.124,NaN
8,16.5,KSF111 + ∆argB,0,0.108,0.002
9,16.5,KSF111 + ∆argB,50,0.119,0.003


In [12]:
fig_arg_response = (
    px
    .bar(
        df_od_mean_sem,
        x='elapsed_time_hr',
        y='od_sample_mean',
        error_y='od_sample_sem',
        color='strain_id',
        facet_col='arg_conc_uM',
        barmode='group',
        labels={
            'arg_conc_uM': '[Arg] µM',
            'od_sample_mean': 'OD600 (mean)',
            'strain_id': 'ADP1 strain',
            'elapsed_time_hr': '',
        },
    )
    .add_traces(
        px.line(
            df_od_mean_sem,
            x='elapsed_time_hr',
            y='od_sample_mean',
            color='strain_id',
            facet_col='arg_conc_uM',
            markers=False,
        )['data']
    )
    .update_layout(
        title='OD ADP1 [KSF111 (∆cphAI ∆astA ∆argBR), KSF001 (wt - pos.ctrl)] | growth in 0µm•Arg after LB preculture and 0µm•Arg wash',
        title_x=0.5,
        showlegend=True,
        boxmode='group',
        boxgap=.8,  # Gap between boxes in the same group (0.5 makes them narrower)
        boxgroupgap=0,  # Gap between different groups of boxes,
        # height=500,
        # width=2000,
        xaxis_title='Elapsed Time [hr]',
        # xaxis_title_x=.5,
    )
    .for_each_trace(dedup_trace_legends())
)

fig_arg_response.write_image('../../figures/pk_exp0011_arg_response_raw.pdf')
fig_arg_response

In [6]:
# let's calculate the max OD600 for each strain at each arginine concentration
df_od_mean_max = df_od_mean_sem.sort_values(
    by=['arg_conc_uM', 'strain_id', 'od_sample_mean'],
).drop_duplicates(
    subset=['arg_conc_uM', 'strain_id'],
    keep='last',
)

# let's remove the background growth from 0•Arg on the auxotrophs (KSF111)
# NOTE this is a bit of a hack, but we do have background growth controls from  EXP010
# TODO confirm that the background growth is in line with the controls from exp010
_bg_KSF001, _bg_KSF111 = df_od_mean_max.loc[lambda df: df.arg_conc_uM == 0].od_sample_mean.iloc[:2]
df_od_mean_max = df_od_mean_max.assign(
    od_sample_mean_sans_bg=lambda df: df.apply(
        lambda r: r.od_sample_mean - (_bg_KSF111 if '111' in r.strain_id else 0),
        axis='columns'
    ),
)

df_od_mean_max

,elapsed_time_hr,strain_id,arg_conc_uM,od_sample_mean,od_sample_sem,od_sample_mean_sans_bg
46,64.5,KSF001 wt,0,1.048,NaN,1.048
40,64.5,KSF111 + ∆argB,0,0.267,0.007,0.000
41,64.5,KSF111 + ∆argB,50,0.715,0.007,0.448
42,64.5,KSF111 + ∆argB,100,0.768,0.008,0.501
43,64.5,KSF111 + ∆argB,150,0.749,0.013,0.482
44,64.5,KSF111 + ∆argB,200,0.723,0.009,0.456
47,64.5,KSF001 wt,300,1.480,NaN,1.480
45,64.5,KSF111 + ∆argB,300,0.718,0.014,0.451


In [7]:
def od_by_arg_conc(
        arg_conc,
        od600_to_cdw=.33 * units.g / units.L,
        perc_cgp_of_cdw=.25,
        perc_arg_of_cgp=.60,
        mw_arg=174.2 * units.g / units.mol,
) -> float:
    od = arg_conc / (od600_to_cdw * perc_cgp_of_cdw * perc_arg_of_cgp / mw_arg)
    return od.to_base_units()


df_cgp_arg = pd.DataFrame(data=dict(arg_conc_µM=[0, 300])).assign(
    od=lambda df: df.arg_conc_µM.apply(lambda c: od_by_arg_conc(c * units.uM).magnitude)
)
df_cgp_arg

,arg_conc_μM,od
0,0,0.000000
1,300,1.055758


In [8]:
def Y_od_from_arg_theoretical(
        od600_to_cdw=.33 * units.g / units.L,
        abundance_arg_of_cdw=.025,  # (1prot/2cdw)(1arg/20prot) = 1/40
        mw_arg=174.2 * units.g / units.mol
) -> float:
    od = arg_conc / (od600_to_cdw * perc_arg_of_cdw / mw_arg)
    return od.to_base_units()


df_arg_cdw = pd.DataFrame(data=dict(arg_conc_µM=[0, 100, 300])).assign(
    od=lambda df: df.arg_conc_µM.apply(lambda c: max_od_by_arg_conc(c * units.uM).magnitude)
)
df_arg_cdw

,arg_conc_μM,od
0,0,0.000000
1,100,2.111515
2,300,6.334545


In [9]:
df_od_mean_max.loc[lambda df: df.strain_id.str.contains('001')]

,elapsed_time_hr,strain_id,arg_conc_uM,od_sample_mean,od_sample_sem,od_sample_mean_sans_bg
46,64.5,KSF001 wt,0,1.048,NaN,1.048
47,64.5,KSF001 wt,300,1.480,NaN,1.480


In [10]:
px.bar(
    df_od_mean_max.loc[lambda df: df.strain_id.str.contains('001')],
    x='arg_conc_uM',
    # y='od_sample_mean',
    y='od_sample_mean_sans_bg',
    error_y='od_sample_sem',
    color='strain_id',
    barmode='group',
    labels={
        'arg_conc_uM': '[Arg] µM',
        'od_sample_mean': 'OD600_mean',
        'od_sample_mean_sans_bg': 'OD600_mean (background removed)',
        'strain_id': 'ADP1 strain',
        'elapsed_time_hr': 'Elapsed Time (hr)',
    },
).update_layout(
    # title='OD ADP1 [KSF107 (∆astA ∆argB), KSF111 (∆astA ∆argBR)] | growth in dosed Arg after preculture and wash',
    # title_x=0.5,
    showlegend=True,
    height=500,
)

In [11]:
fig_yield = px.scatter(
    df_od_mean_max,
    x='arg_conc_uM',
    # y='od_sample_mean',
    y='od_sample_mean_sans_bg',
    color='strain_id',
    labels={
        'strain_id': 'ADP1 Strain',
        'od_sample_x': 'OD',
        'arg_conc_μM': '[Arg] (µM)'
    },
    trendline='rolling',
    trendline_options=dict(function='median', window=1),
    # trendline='ols',
    # trendline='expanding',
    # trendline_options=dict(function='max'),
).update_layout(
    # title='ADP1 {KSF230 ∆argB, KSF? ∆argB ∆argR} Biomass Yield Curves',
    title_x=0.5,
)
fig_yield.add_traces([
    # constraint
    go.Scatter(
        x=df_arg_cdw.arg_conc_μM, y=df_arg_cdw.od,
        name='ODmax by 2.5% Arg',
        mode='lines', line=dict(dash='dash'),
        # fill='tonexty',
    ),
])
# yield_fits = px.get_trendline_results(fig_yield).set_index('ADP1 Strain').px_fit_results

fig_yield.update_layout(template='simple_white')

# fig_yield.write_image('../../figures/pk_exp007_ArginineTitrationResponse.pdf')
# fig_yield

In [10]:
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly_dark'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [11]:
yield_fits['KSF230'].summary()
yield_fits['∆argR'].summary()


NameError: name 'yield_fits' is not defined

In [ ]:
def theor_arg_from_OD600_with_CGP(od600: float, frac_cgp_of_cdw: float = 0.25, frac_arg_of_cgp: float = .6,
                                  mw_arg: float = 174.2) -> float:
    return od600 * .33 * frac_cgp_of_cdw * frac_arg_of_cgp / mw_arg * 1E6


def theor_OD600_with_CGP_for_x_arg(x_arg: float, frac_cgp_of_cdw: float = 0.25, frac_arg_of_cgp: float = .6,
                                   mw_arg: float = 174.2) -> float:
    return x_arg / (.33 * frac_cgp_of_cdw * frac_arg_of_cgp / mw_arg * 1E6)


def yield_in_g_g(x):
    return x * .33 / (1.742 * 10 ** -4)


yield_in_g_g(0.0004)

theor_arg_from_OD600_with_CGP(1)
theor_OD600_with_CGP_for_x_arg(100)
theor_OD600_with_CGP_for_x_arg(250)
# t = yield_fits['KSF230'].summary().tables[1]
# t[('x1')]

In [ ]:
.506 / theor_OD600_with_CGP_for_x_arg(100, frac_cgp_of_cdw=.5)

In [ ]:
import pandera as pa
import pandera.typing as pat
from analysis.schemas import SakaguchiODSchema


@pa.check_types()
def sakaguchi_analysis(df: pat.DataFrame[SakaguchiODSchema]):
    return df

In [ ]:
# noinspection PyTypeChecker
df_sakaguchi = pd.read_excel(
    '~/tyo_lab_pk/04 Raw Data/EXP_0007_PK_20240909_ArgConcentrationResponse.xlsx',
    sheet_name='Sakaguchi for residual arg',
    usecols='B:E',
    skiprows=45,
)

sakaguchi_analysis(df_sakaguchi)

In [ ]:
plotly_colors = {
    "SP_Brights_Green": "rgb(81, 255, 0)",
    "SP_Brights_Light_Blue": "rgb(132, 255, 199)",
    "SP_Brights_Blue": "rgb(81, 168, 255)",
    "SP_Brights_Yellow": "rgb(229, 255, 28)",
    "SP_Brights_Orange": "rgb(255, 196, 13)",
    "SP_Brights_Orange_Red": "rgb(252, 45, 51)",
    "SP_Dark_Green": "rgb(27, 168, 36)",
    "SP_Dark_Light_Blue": "rgb(35, 150, 197)",
    "SP_Dark_Blue": "rgb(58, 38, 147)",
    "SP_Dark_Yellow": "rgb(209, 219, 0)",
    "SP_Dark_Orange": "rgb(201, 126, 19)",
    "SP_Dark_Orange_Red": "rgb(229, 57, 0)"
}

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

black = 'rgba(255,255,255,0)'

# create linear space
x_vals = np.linspace(0, 10, 100)

# doubling references
y_vals_1_1 = x_vals
y_vals_1_2 = 2 * x_vals
y_vals_1_4 = 4 * x_vals

fig.add_trace(
    go.Scatter(x=x_vals, y=y_vals_1_1, mode='lines', line=dict(dash='dash', color='black', width=1), name='1:1'))
fig.add_trace(
    go.Scatter(x=x_vals, y=y_vals_1_2, mode='lines', line=dict(dash='dash', color='black', width=1), name='1:2'))
fig.add_trace(
    go.Scatter(x=x_vals, y=y_vals_1_4, mode='lines', line=dict(dash='dash', color='black', width=1), name='1:4'))

# x -> 1:1   real bad like
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, x_vals[::-1]]),
                         y=np.concatenate([y_vals_1_1, np.zeros_like(x_vals)]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Brights_Orange_Red'],
                         opacity=0.2,
                         line=dict(color=black),
                         showlegend=True))
# 1:1 -> 1:2  meh
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, x_vals[::-1]]),
                         y=np.concatenate([y_vals_1_1, y_vals_1_2[::-1]]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Dark_Orange_Red'],
                         opacity=0.2,
                         line=dict(color=black),
                         showlegend=False))
# 1:2 -> 1:4  ok
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, x_vals[::-1]]),
                         y=np.concatenate([y_vals_1_2, y_vals_1_4[::-1]]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Dark_Green'],
                         opacity=0.2,
                         line=dict(color=black),
                         showlegend=False))
# y -> 1:4  cookin
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, np.zeros_like(x_vals)]),
                         y=np.concatenate([y_vals_1_4, y_vals_1_4[::-1]]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Brights_Green'],
                         line=dict(color=black),
                         opacity=0.2,
                         showlegend=False))

fig.add_annotation(x=.9, y=.8, text="death", showarrow=False, font=dict(size=20))
fig.add_annotation(x=.8, y=.95, text="no doubling", showarrow=False, font=dict(size=20))
fig.add_annotation(x=.35, y=.95, text="1-2 doublings", showarrow=False, font=dict(size=20))
fig.add_annotation(x=.11, y=.95, text="≥2 doublings", showarrow=False, font=dict(size=20))

# theoretical and experimental curves

fig.add_trace(go.Scatter(x=x_vals, y=1.43 * x_vals, mode='lines', line=dict(color='black'), name='∆argBR Best'))
fig.add_trace(go.Scatter(x=x_vals, y=4.6 * x_vals, mode='lines', line=dict(color='purple'), name='theoretical'))

fig.add_annotation(x=.2, y=.51, text="Y theoretical = 4.5", showarrow=False, font=dict(size=16))
fig.add_annotation(x=.42, y=.4, text="Y ∆argBR (best of) = 1.5", showarrow=False, font=dict(size=16))

# Update axes and layout for the plot
fig.update_layout(
    xaxis_title='ADP1 with CGP from Phase 1 (OD600)',
    yaxis_title='ADP1 in Phase 2 (OD600)',
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    showlegend=False,
    width=700,
    height=700,
    plot_bgcolor='rgba(0,0,0,0)'
)

fig

In [ ]:
fig.write_image('/Users/pasha/src/nu/posters/artifacts/yield_cgp_for_arg.pdf')


### Sakaguchi on residual Arginine

In [ ]:
df_od_saka = (
    pd.read_excel(
        '/Users/pasha/onedrive/nu/Katsev Pavel/04 Raw Data/EXP_0011_PK_20241029_ArgConcentrationResponse.xlsx',
        sheet_name='Sakaguchi for residual arg',
        usecols='B:D',
        skiprows=61,
    )
    .assign(arg_conc=lambda df: df.condition.str.split('•').str.get(0).apply(units.parse_expression))
    .assign(arg_conc_uM=lambda df: df.arg_conc.apply(lambda qty: int(qty.to('micromolar').magnitude)))
    # .assign(lim=lambda df: np.abs((df.od_sample - df.od_sample.mean()) / df.od_sample.std(ddof=0)) < 3)
)
df_od_saka